In [33]:
import nltk
from collections import Counter
import glob
import os
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Paths for all the different corpora
folderpaths = ['../AnxietyMeditations/', '../SleepMeditations/',
               '../LearningAndGrowthMeditations/', '../MorningMeditations/']

#array to store the text of each file 
corpus = []

for folder in folderpaths:
    if folder == '../AnxietyMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
    elif folder == '../SleepMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
    elif folder == '../LearningAndGrowthMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
    elif folder == '../MorningMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)

newStopWords = ['[', ']', '`', '#', '$',
                '(', ')', "Music", ':', "....", "--", ",", "''", '`', '"']
stopwords = set(stopwords.words('english') +
                newStopWords + list(string.punctuation))

# Tokenizing by word and filtering stopwords
all_tokens = [w for w in word_tokenize(text.lower()) if not w in stopwords]



In [34]:
import nltk
from nltk.tokenize import word_tokenize

# Preprocess the text data
tokenized_corpus = []
for text in corpus:
    # Tokenize and lowercase
    tokens = [w for w in word_tokenize(text.lower()) if not w in stopwords]
    tokenized_corpus.append(tokens)


In [35]:
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
# Preprocess the text data
tokenized_corpus = []
for text in corpus:
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    tokenized_corpus.append(tokens)

context_windows = []
window_size = 4  # Size of the context window

for sentence in tokenized_corpus:
    sentence_length = len(sentence)
    for i in range(sentence_length):
        # Extract the target word
        target_word = sentence[i]

        # Extract the context words within the window size
        context_words = []
        for j in range(max(0, i - window_size), min(sentence_length, i + window_size + 1)):
            if j != i:
                context_words.append(sentence[j])

        # Append the target word and its context words as a tuple
        # Convert context_words to tuple
        context_windows.append((target_word, tuple(context_words)))

# Train the Word2Vec model
model = Word2Vec(context_windows, window=5,
                 min_count=10, sg=0)  # CBOW approach

# Accessing trained word vectors
word_vector = model.wv['breathe']  # Get the word vector for a specific word
print(word_vector)

[-0.04357344  0.06469832  0.00708946  0.10132951  0.04842198 -0.04448317
  0.09263577  0.04414168 -0.10461225  0.00455453 -0.01399897 -0.09782488
  0.01634117 -0.04093378  0.01624235  0.04146756  0.00205488 -0.07824838
 -0.02845545 -0.22111169  0.00766507  0.08452726  0.09406016 -0.06352351
 -0.02355131  0.09575079 -0.05538119 -0.01975208 -0.1269004   0.01042841
  0.08245099  0.03363612  0.04226923 -0.00592274 -0.11145383  0.09136933
 -0.01363419  0.00932562 -0.04875571 -0.12297434  0.09713392 -0.02909073
 -0.01400315 -0.05426046  0.07613517 -0.03536762 -0.06918352  0.05822345
  0.09930514  0.01208888 -0.01047047 -0.02739201  0.01334653  0.01012277
 -0.07499318  0.05187471  0.00415072 -0.07271872 -0.04003773 -0.03871902
  0.02842201  0.04550796  0.06908636  0.02626394 -0.03112886  0.02367999
 -0.10433532  0.02260037 -0.0981443  -0.01794084  0.00393988  0.08520032
  0.0048525  -0.03539305  0.01660635 -0.01464625  0.0289981  -0.08286708
 -0.04124979 -0.04354507  0.0039212  -0.0758892  -0

In [36]:
import os
import nltk
from gensim.models import Word2Vec

# Define the directory containing the text files
directory = "./txt Files/"

all_vectors = { 'sight': [], 'smell': [], 'sound': [], 'taste': []
, 'touch': []}

# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r") as f:
            text = f.read()
            words = nltk.word_tokenize(text)
            for word in words:
                try:
                    word_vector = model.wv[word.lower()]
                    all_vectors[filename[:-4]].append(word_vector)
                    # print(f"Word: {word}\nVector: {word_vector}\n")
                except KeyError:
                    continue
                    # print(f"Word '{word}' not in vocabulary.\n")

# print(len(all_vectors['sight']))
# print(len(all_vectors['sight']))


In [38]:
import numpy as np
from scipy.spatial.distance import cosine

for key in all_vectors.keys(): 
    # Calculate the pairwise cosine similarity distance between each row in the matrix
    sight_len = len(all_vectors[key])
    distances = np.zeros((sight_len, sight_len))
    for i in range(sight_len):
        for j in range(i+1, sight_len):
            distances[i][j] = cosine(all_vectors[key][i], all_vectors[key][j])
            distances[j][i] = distances[i][j]

    # Print the resulting distance matrix
    distances = distances[:15, :15]
    print(distances.shape)

# Normalize the matrix to the range 0-1
# normalized_distance = (distances - np.min(distances)) / (np.max(distances) - np.min(distances))

(15, 15)
(15, 15)
(15, 15)
(15, 15)
(15, 15)
